In [ ]:
import requests
import os
import time

class SoilPropertyFetcher:
    def __init__(self, lat, lon):
        self.lat = lat
        self.lon = lon
        self.rest_url = "https://rest.isric.org"
        self.prop_query_url = f"{self.rest_url}/soilgrids/v2.0/properties/query"


    def fetch_property(self, property_name, retries=3, backoff_factor=2):
        self.property_name = property_name
        params = {
            "lat": self.lat,
            "lon": self.lon,
            "property": self.property_name,
            "depth": "5-15cm",
            "value": "mean"
        }

        for i in range(retries):
            response = requests.get(self.prop_query_url, params=params)
            if response.status_code == 200:
                data = response.json()
                mean_value = data['properties']['layers'][0]['depths'][0]['values']['mean']
                time.sleep(12)  # Respect API rate limits
                return mean_value / 10  # Scaling the mean value appropriately


            elif response.status_code == 429:
                wait_time = (backoff_factor ** i) * 10  # Exponential backoff
                print(f"Rate limit exceeded. Retrying in {wait_time} seconds...")
                time.sleep(wait_time)
            else:
                raise Exception(f"Error fetching data: {response.status_code} - {response.text}")



        # If all retries fail
        raise Exception(f"Failed to fetch data after {retries} retries due to rate limiting.")


point_1 = SoilPropertyFetcher(42,24)
print(point_1.fetch_property("clay"))

properties = ['clay', 'sand', 'silt']

11.1


In [ ]:
import requests
import time
from urllib.parse import urlencode

class SoilPropertyFetcher:
    def __init__(self, lat, lon):
        self.lat = lat
        self.lon = lon
        self.base_url = "https://rest.isric.org/soilgrids/v2.0/properties/query"

    def construct_url(self, properties, depth="5-15cm", value="mean"):
        # Create the query parameters dynamically
        query_params = {
            "lat": self.lat,
            "lon": self.lon,
            "depth": depth,
            "value": value
        }

        # Add each property as a separate query parameter
        # This makes sure that multiple properties are added as `property=...` repeatedly
        properties_params = [('property', prop) for prop in properties]

        # Combine the query parameters with properties
        query_string = urlencode(query_params) + '&' + urlencode(properties_params, doseq=True)

        # Create the full URL by appending the query string to the base URL
        full_url = f"{self.base_url}?{query_string}"
        return full_url

    def fetch_properties(self, properties, retries=3, backoff_factor=2):
        results = {}
        full_url = self.construct_url(properties)

        for i in range(retries):
            response = requests.get(full_url, headers={'accept': 'application/json'})
            if response.status_code == 200:
                data = response.json()
                print(data)  # Debugging: Print the response

                try:
                    # Loop over each property and extract the mean value
                    for property_name in properties:
                        mean_value = None
                        for layer in data['properties']['layers']:
                            print(f"Layer found: {layer['name']}")
                            if layer['name'] == property_name:
                                mean_value = layer['depths'][0]['values'].get('mean', None)
                                if mean_value is not None:
                                    mean_value = mean_value / 10
                                break
                        if mean_value is None:
                            print(f"No data found for property: {property_name}")
                        results[property_name] = mean_value

                    time.sleep(12)
                    break

                except (KeyError, IndexError):
                    print(f"Error extracting data for properties: {properties}")
                    return None

            elif response.status_code == 429:
                wait_time = (backoff_factor ** i) * 10
                print(f"Rate limit exceeded. Retrying in {wait_time} seconds...")
                time.sleep(wait_time)
            else:
                raise Exception(f"Error fetching data: {response.status_code} - {response.text}")

        else:
            raise Exception(f"Failed to fetch data after {retries} retries due to rate limiting.")

        return results


# Example usage
lat = 50
lon = 24
properties = ['clay', 'nitrogen', 'sand', 'silt', 'soc']

soil_fetcher = SoilPropertyFetcher(lat, lon)
results = soil_fetcher.fetch_properties(properties)

# Print the results for each property
for property_name, value in results.items():
    if value is not None:
        print(f"{property_name}: {value}")
    else:
        print(f"{property_name}: No data found")


{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [24.0, 50.0]}, 'properties': {'layers': [{'name': 'clay', 'unit_measure': {'d_factor': 10, 'mapped_units': 'g/kg', 'target_units': '%', 'uncertainty_unit': ''}, 'depths': [{'range': {'top_depth': 5, 'bottom_depth': 15, 'unit_depth': 'cm'}, 'label': '5-15cm', 'values': {'mean': 249}}]}, {'name': 'nitrogen', 'unit_measure': {'d_factor': 100, 'mapped_units': 'cg/kg', 'target_units': 'g/kg', 'uncertainty_unit': ''}, 'depths': [{'range': {'top_depth': 5, 'bottom_depth': 15, 'unit_depth': 'cm'}, 'label': '5-15cm', 'values': {'mean': 181}}]}, {'name': 'sand', 'unit_measure': {'d_factor': 10, 'mapped_units': 'g/kg', 'target_units': '%', 'uncertainty_unit': ''}, 'depths': [{'range': {'top_depth': 5, 'bottom_depth': 15, 'unit_depth': 'cm'}, 'label': '5-15cm', 'values': {'mean': 261}}]}, {'name': 'silt', 'unit_measure': {'d_factor': 10, 'mapped_units': 'g/kg', 'target_units': '%', 'uncertainty_unit': ''}, 'depths': [{'range': {'top_